In [ ]:
pip install dice_ml

In [ ]:
# HIDE
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from helpers.fairness.helpers import plot_distributions


import dice_ml

import os


sns.set(style="white", palette=[sns.color_palette('muted')[i] for i in [0,2]], 
        color_codes=True, context="talk")

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

# import keras as ke
# import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

create_gif = False
# supress deprecation warnings from TF
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print(f"sklearn: {sk.__version__}")
print(f"pandas: {pd.__version__}")
print(f"tensorflow: {tf.__version__}")

In [ ]:
from helpers.mlp_prediction import *

datasetX, dataset ,target, numerical, categorical = load_and_preprocess_data('/Users/nilsness/Desktop/Uni Skripte und Folien/Masterarbeit/Programming/Loan-Approval-Prediction.csv')

numerical.pop(-1)
categorical.append('Credit_History')

print(numerical, categorical)

In [ ]:
dataset.shape
dataset.Gender.value_counts()

In [ ]:
# Calculate the number of instances to remove
num_instances = int(0.75 * len(dataset[(dataset['Gender'] == 'Female') & (dataset['Loan_Status'] == 'Y')]))

# Get the indices of the instances to remove
indices_to_remove = dataset[(dataset['Gender'] == 'Female') & (dataset['Loan_Status'] == 'Y')].sample(num_instances).index

# Remove the instances from the dataset
dataset = dataset.drop(indices_to_remove)

target = dataset['Loan_Status']
datasetX = dataset.drop(columns=['Loan_Status'])
dataset


In [ ]:
datasetX.Gender.value_counts()

In [ ]:
x_train_encoded, x_test_encoded, y_train, y_test, Z_train, Z_test, datasetX1, target,Z, dataset_num_headers, scaler, encoder = do_it_like_numbers_do(datasetX,target,numerical, categorical)
models, model_list, transformations = build_pipeline(numerical,categorical)

In [ ]:
datasetL = dataset.copy()
dataset.drop(columns='Loan_Status',inplace=True)

In [ ]:
dataset_num_headers

In [ ]:
x_train_encoded[categorical] = x_train_encoded[categorical].astype('category')

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=42)

# Train the model
model.fit(x_train_encoded, y_train)

In [ ]:
prediction = model.predict(dataset_num_headers)
dataset['Predictions'] = prediction
dataset_num_headers['Predictions'] = prediction
cat = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Credit_History','Property_Area_Rural','Property_Area_Urban']
dataset_num_headers[cat]=dataset_num_headers[cat].astype('category')
dataset_num_headers



In [ ]:
y_pred = model.predict(x_test_encoded)
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Berechnung der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnung der Präzision
precision = precision_score(y_test, y_pred)

# Berechnung des Recall
recall = recall_score(y_test, y_pred)

# Berechnung des F1-Scores
f1 = f1_score(y_test, y_pred)
# Berechnung der Ks =_(y_test, y_pred)

print("Genauigkeit:", accuracy)
print("Präzision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)



import matplotlib.pyplot as plt


# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
pred = model.predict_proba(dataset_num_headers.drop(columns='Predictions'))
pred = pd.DataFrame(pred, columns=['No', 'Yes'])
pred = pred['Yes']

In [ ]:
dataset_num_headers.drop(columns='Predictions',inplace=True)

In [ ]:
def get_query_instances(new_model,datasetX):

    pred_ = pd.DataFrame()
    for i in range (0, 35):

        pred = new_model.predict_proba(datasetX)
        pred = pd.DataFrame(pred, columns=['Yes','No'])
        pred_['Prediction'+str(i)]= pred['Yes']

    pred_ = pred_.mean(axis=1)
    print(pred_)
    datasetX['Prediction'] = pred_
    datasetX['Loan_Status'] = datasetL['Loan_Status']



    query_instances = datasetX[datasetX['Prediction']<0.5]

 
    query_instances = query_instances.sort_values(by='Prediction', ascending=False)
    query_instances_sparsity = query_instances.copy()
    query_instances_sparsity = query_instances_sparsity[query_instances_sparsity['Gender']==1]
    #print(query_instances)   
    query_instances = query_instances.iloc[[-1,0]]
    

    query_instances_sparsity.drop(columns=['Prediction'], inplace = True)

    query_instances_sparsity.drop(columns=['Loan_Status'], inplace = True)
    query_instances.drop(columns=['Loan_Status'], inplace = True)
    query_instances.drop(columns=['Prediction'], inplace=True)

    num_duplicates = len(query_instances_sparsity) // len(query_instances.iloc[[0]])

    # Duplicate the DataFrame by concatenating it with itself
    duplicated_instances = pd.concat([query_instances.iloc[[0]]] * num_duplicates, ignore_index=True)


    # Trim the duplicated DataFrame to match the length of query_instances_sparsity
    duplicated_instances = duplicated_instances[:len(query_instances_sparsity)]

    from helpers.analysis import sparsity
    sparsity = sparsity(query_instances_sparsity,query_instances_sparsity, duplicated_instances)

  
    while len(query_instances) < 3:

        sparse_index = list(sparsity.keys())[list(sparsity.values()).index(min(sparsity.values()))]

        print(sparse_index)
        print(query_instances_sparsity.iloc[[sparse_index]])
    
        if new_model.predict(query_instances_sparsity.iloc[[sparse_index]])[0] < 0.5:
            query_instances = query_instances.append(query_instances_sparsity.iloc[[sparse_index]])
        else:
            query_instances_sparsity.drop(query_instances_sparsity.index[sparse_index], inplace=True)
    
    query_instances = query_instances.drop_duplicates()

    factuals = query_instances.copy()
    


    #query_instances.drop(columns=['Loan_Status'], inplace=True)
   
    datasetX.drop(columns=['Prediction'],inplace=True)
    datasetX.drop(columns=['Loan_Status'],inplace=True)
    
    return query_instances, factuals

In [ ]:
query_instances , factuals = get_query_instances(model,dataset_num_headers)

In [ ]:
query_instances

In [ ]:
model.predict(query_instances)

In [ ]:
model.predict_proba(query_instances)

In [ ]:
dataset_num_headers['Loan_Status'] = target
dataset_num_headers.info()

In [ ]:
dataset_num_headers [cat] = dataset_num_headers[cat].astype('float')

In [ ]:
backend = 'sklearn'

# Step 2: dice_ml.Model
m = dice_ml.Model(model=model, backend=backend) 

In [ ]:
datasetX['Loan_Status'] = target
d = dice_ml.Data(dataframe=dataset_num_headers, continuous_features=['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
                 , outcome_name='Loan_Status', feature_names=list(dataset_num_headers.columns), categorical_features=cat,
                 )


In [ ]:
# initiate DiCE
exp_random = dice_ml.Dice(d, m, method="random")

In [ ]:
query_instances[cat] = query_instances[cat].astype('float')

query_instances

In [ ]:
# generate counterfactuals
dice_exp_random = exp_random.generate_counterfactuals(query_instances, total_CFs=1, desired_class="opposite", verbose=True,
                                                    proximity_weight=100.0, diversity_weight=0.2, random_seed=42,
                                                    stopping_threshold=0.5)

In [ ]:
dice_exp_random.visualize_as_dataframe(show_only_changes=True)

In [ ]:
dice_exp_recourse = exp_random.generate_counterfactuals(query_instances, total_CFs=1, desired_class="opposite", verbose=True,
                                                        features_to_vary=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History'],
                                                       proximity_weight=1.5, diversity_weight=0.2, categorical_penalty=0.5, random_seed=42, 
                                                       sample_size=480, stopping_threshold=0.5, posthoc_sparsity_param=0.5)

In [ ]:
dice_exp_recourse.visualize_as_dataframe(show_only_changes=True)

In [ ]:
counterfactuals = dice_exp_recourse.to_json()

In [ ]:
counterfactuals

In [ ]:
import json
counterfactuals = json.loads(counterfactuals)

cfs_list = counterfactuals['cfs_list']

cfs_1 = pd.DataFrame(cfs_list[0], columns=['Gender', 'Married','Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 
                                            'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History','Property_Area_Rural' ,'Property_Area',
                                            'Property_Area_Urban'
                                            , 'Loan_Status'])
cfs_2= pd.DataFrame(cfs_list[1], columns=['Gender', 'Married','Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 
                                            'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History','Property_Area_Rural' ,'Property_Area',
                                            'Property_Area_Urban'
                                            , 'Loan_Status'])
cfs_3= pd.DataFrame(cfs_list[2], columns=['Gender', 'Married','Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 
                                            'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History','Property_Area_Rural' ,'Property_Area',
                                            'Property_Area_Urban'
                                            , 'Loan_Status'])

In [ ]:
# Assuming `scaler` is the MinMaxScaler object used to scale the data
rescaled_query_instances = scaler.inverse_transform(query_instances[numerical])
rescaled_query_instances


In [ ]:
query_instances_rescaled = query_instances.copy()
query_instances_rescaled[numerical] = rescaled_query_instances
query_instances_rescaled

In [ ]:
cfs_1 = pd.DataFrame([cfs_1], columns=dataset_num_headers.columns)
cfs_2 = pd.DataFrame([cfs_2], columns=dataset_num_headers.columns)
cfs_3 = pd.DataFrame([cfs_3], columns=dataset_num_headers.columns)

In [ ]:
cfs_1_rescaled = cfs_1.copy()
cfs_2_rescaled = cfs_2.copy()
cfs_3_rescaled = cfs_3.copy()

rescaled_cfs_1 = scaler.inverse_transform(cfs_1[numerical])
rescaled_cfs_2 = scaler.inverse_transform(cfs_2[numerical])
rescaled_cfs_3 = scaler.inverse_transform(cfs_3[numerical])

cfs_1_rescaled[numerical] = rescaled_cfs_1
cfs_2_rescaled[numerical] = rescaled_cfs_2
cfs_3_rescaled[numerical] = rescaled_cfs_3

cfs_1_rescaled

In [ ]:
cfs_2_rescaled

In [ ]:
cfs_3_rescaled

In [ ]:
counter = pd.DataFrame(cfs_1.iloc[0]).transpose()
counter = counter.append(cfs_2.iloc[0])
counter = counter.append(cfs_3.iloc[0])

counter

In [ ]:
counter_rescaled = counter.copy()
counter_rescaled[numerical] = scaler.inverse_transform(counter[numerical])

In [ ]:
expl_random_locimp = dice_ml.Dice(d, m, method="random")

In [ ]:
feature_names = dataset_num_headers.drop(columns='Loan_Status').columns
feature_names.values

In [ ]:
import lime
import lime.lime_tabular
explainer = lime.lime_tabular.LimeTabularExplainer(dataset_num_headers.drop(columns='Loan_Status').values, mode="classification",categorical_features=[0,1,2,3,4,9,10,11,12], 
                                                   training_labels=datasetX['Loan_Status'], feature_names=feature_names)

In [ ]:
def predict_fn (data):
    return np.array(list(zip(1-model.predict(data),model.predict(data))))

In [ ]:
exp1 = explainer.explain_instance(query_instances.iloc[0,:].values, predict_fn,labels=(0,1) ,num_features=13)
exp2 = explainer.explain_instance(query_instances.iloc[1,:].values, predict_fn,labels=(0,1), num_features=13)
exp3 = explainer.explain_instance(query_instances.iloc[2,:].values, predict_fn,labels=(0,1), num_features=13)

In [ ]:
exp1.show_in_notebook(show_table=True, show_all=False)
exp2.show_in_notebook(show_table=True, show_all=False)  
exp3.show_in_notebook(show_table=True, show_all=False)

In [ ]:
explanation_1 = pd.DataFrame(exp1.as_list(),columns=['Feature','Contribution'])
explanation_2 = pd.DataFrame(exp2.as_list(),columns=['Feature','Contribution'])
explanation_3 = pd.DataFrame(exp3.as_list(),columns=['Feature','Contribution'])
print(explanation_1)
print(explanation_2)
print(explanation_3)

In [ ]:
from helpers.evaluate_distance import *
from helpers.evaluate_redundancy import *



In [ ]:
counter = pd.DataFrame(cfs_1.iloc[0]).transpose()
counter = counter.append(cfs_2.iloc[0])
counter = counter.append(cfs_3.iloc[0])

counter.reset_index(drop=True, inplace=True)
counter


In [ ]:
counter[numerical] = scaler.transform(counter[numerical])

In [ ]:
counter

In [ ]:
factuals = query_instances
factuals.reset_index(drop=True, inplace=True)
factuals['Loan_Status'] = [0,0,0]
factuals

In [ ]:
df_cf = counter.copy().drop(columns=['Loan_Status'])
df_fc = factuals.copy().drop(columns=['Loan_Status'])
index = df_fc.index 
df_cf.index = index

categorical.append('Credit_History')
categorical.append('Property_Area')
numerical = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']

In [ ]:
numerical

In [ ]:
from helpers.analysis import *
dist = Distance()

distances = dist.get_evaluation(factuals,counter)

sparsity = sparsity(counter,counter,factuals)

validity = validity_total(counter,factuals,model)

check_binary_cat = check_binary_categorical(counter, categorical)

check_one_hot = check_one_hot_integrity(counter,categorical)

mean_absolute_deviation_dist = madd(df_fc, df_cf,numerical,categorical,df_cf,df_fc)

In [ ]:
mean_absolute_deviation_dist

In [ ]:
print(check_binary_cat)
print(check_one_hot)

In [ ]:
distances

In [ ]:
sparsity

In [ ]:
validity

In [ ]:
mahalanobis_dist = md(factuals, df_cf, df_cf, factuals)
mahalanobis_dist